# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [65]:
import random

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def value(self):
        if self.rank in ['10', 'J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        else:
            return int(self.rank)

    def __str__(self):
        return f"{self.rank}{self.suit}"

class Deck:
    def __init__(self, num_decks=6):
        suits = ['♠', '♥', '♦', '♣']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.cards = [Card(suit, rank) for _ in range(num_decks) for suit in suits for rank in ranks]
        random.shuffle(self.cards)
        self.plastic_card_position = random.randint(15, 30)  # Random position for the plastic card

    def draw(self):
        if len(self.cards) < self.plastic_card_position:
            self.reshuffle()
        return self.cards.pop()

    def reshuffle(self):
        print("Reshuffling the deck...")
        self.__init__(len(self.cards) // 52)  # Reinitialize the deck with the same number of decks
        random.shuffle(self.cards)

class Player:
    def __init__(self, name, chips):
        self.name = name
        self.chips = chips
        self.hand = []

    def bet(self, amount):
        if amount <= self.chips:
            self.chips -= amount
            return amount
        else:
            print(f"{self.name} does not have enough chips to bet {amount}.")
            return 0

    def win(self, amount):
        self.chips += amount

    def add_card(self, card):
        self.hand.append(card)

    def clear_hand(self):
        self.hand = []

    def hand_value(self):
        return calculate_hand_value(self.hand)

    def __str__(self):
        return f"{self.name}: {' '.join(map(str, self.hand))} (Value: {self.hand_value()})"

class BlackjackGame:
    def __init__(self, num_decks=6):
        self.deck = Deck(num_decks)
        self.players = []
        self.dealer_hand = []

    def add_player(self, player):
        self.players.append(player)

    def deal_initial_cards(self):
        for player in self.players:
            player.add_card(self.deck.draw())
            player.add_card(self.deck.draw())

        # Dealer's hand: one card face up, one card face down
        self.dealer_hand = [self.deck.draw(), self.deck.draw()]

    def play_round(self):
        self.deal_initial_cards()
        print("\nInitial hands:")
        for player in self.players:
            print(player)
        print(f"Dealer: {self.dealer_hand[0]} ?")

        for player in self.players:
            while player_strategy(player, self.dealer_hand[0]) == 'hit':
                player.add_card(self.deck.draw())
                print(player)
                if player.hand_value() > 21:
                    print(f"{player.name} busts!")
                    break

        # Reveal dealer's hole card
        print(f"\nDealer's hand: {' '.join(map(str, self.dealer_hand))} (Value: {calculate_hand_value(self.dealer_hand)})")

        # Dealer plays according to standard rules
        self.dealer_play()

        # Determine outcome for each player
        for player in self.players:
            outcome = determine_winner(player, self.dealer_hand)
            if outcome == "Player":
                print(f"{player.name} wins!")
                player.win(2)  # Assuming a bet of 1 chip
            elif outcome == "Dealer":
                print(f"Dealer wins against {player.name}.")
            else:
                print(f"{player.name} pushes (ties).")
                player.win(1)  # Return the bet

        # Clear hands for the next round
        for player in self.players:
            player.clear_hand()
        self.dealer_hand = []

    def dealer_play(self):
        while calculate_hand_value(self.dealer_hand) < 17:
            self.dealer_hand.append(self.deck.draw())
            print(f"Dealer hits: {' '.join(map(str, self.dealer_hand))} (Value: {calculate_hand_value(self.dealer_hand)})")
            if calculate_hand_value(self.dealer_hand) > 21:
                print("Dealer busts!")

def player_strategy(player, dealer_up_card):
    """
    Basic strategy for the player:
    - Hit if hand value is less than 17.
    - Stand otherwise.
    """
    if player.hand_value() < 17:
        return 'hit'
    else:
        return 'stand'

def calculate_hand_value(hand):
    value = sum(card.value() for card in hand)
    aces = sum(1 for card in hand if card.rank == 'A')
    while value > 21 and aces:
        value -= 10
        aces -= 1
    return value

def determine_winner(player, dealer_hand):
    player_value = player.hand_value()
    dealer_value = calculate_hand_value(dealer_hand)

    if player_value > 21:
        return "Dealer"
    elif dealer_value > 21 or player_value > dealer_value:
        return "Player"
    elif player_value < dealer_value:
        return "Dealer"
    else:
        return "Push"  # Indicates a tie

# Example usage
if __name__ == "__main__":
    game = BlackjackGame()
    player1 = Player("Isihack", 100)
    player2 = Player("Abdul", 100)
    game.add_player(player1)
    game.add_player(player2)

    while True:
        game.play_round()
        if input("Play another round? (y/n): ").lower() != 'y':
            break



Initial hands:
Isihack: 9♥ 8♦ (Value: 17)
Abdul: A♦ 4♦ (Value: 15)
Dealer: J♠ ?
Abdul: A♦ 4♦ 9♣ (Value: 14)
Abdul: A♦ 4♦ 9♣ 6♠ (Value: 20)

Dealer's hand: J♠ 9♠ (Value: 19)
Dealer wins against Isihack.
Abdul wins!


Play another round? (y/n):  y



Initial hands:
Isihack: 6♠ J♥ (Value: 16)
Abdul: 7♥ 6♥ (Value: 13)
Dealer: Q♣ ?
Isihack: 6♠ J♥ 8♥ (Value: 24)
Isihack busts!
Abdul: 7♥ 6♥ 3♣ (Value: 16)
Abdul: 7♥ 6♥ 3♣ K♠ (Value: 26)
Abdul busts!

Dealer's hand: Q♣ 10♥ (Value: 20)
Dealer wins against Isihack.
Dealer wins against Abdul.


Play another round? (y/n):  y



Initial hands:
Isihack: 7♦ J♠ (Value: 17)
Abdul: 2♦ 5♥ (Value: 7)
Dealer: J♣ ?
Abdul: 2♦ 5♥ Q♠ (Value: 17)

Dealer's hand: J♣ 8♦ (Value: 18)
Dealer wins against Isihack.
Dealer wins against Abdul.


Play another round? (y/n):  y



Initial hands:
Isihack: 6♥ 9♥ (Value: 15)
Abdul: 4♦ K♦ (Value: 14)
Dealer: A♣ ?
Isihack: 6♥ 9♥ 2♣ (Value: 17)
Abdul: 4♦ K♦ A♠ (Value: 15)
Abdul: 4♦ K♦ A♠ 2♣ (Value: 17)

Dealer's hand: A♣ Q♦ (Value: 21)
Dealer wins against Isihack.
Dealer wins against Abdul.


Play another round? (y/n):  n


2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

In [68]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def value(self):
        pass

    def __str__(self):
        pass

class Deck:
    def __init__(self, num_decks=6):
        self.cards = []

    def draw(self):
        pass

    def shuffle(self):
        pass

class Player:
    def __init__(self, name, chips):
        self.name = name
        self.chips = chips
        self.hand = []

    def bet(self, amount):
        pass

    def win(self, amount):
        pass

    def add_card(self, card):
        pass

    def clear_hand(self):
        pass

class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        pass

    def clear(self):
        pass

class BlackjackGame:
    def __init__(self, num_decks=6):
        self.deck = Deck(num_decks)
        self.players = []
        self.dealer_hand = []

    def add_player(self, player):
        pass

    def deal_initial_cards(self):
        pass

    def play_round(self):
        pass

    def dealer_play(self):
        pass

def player_strategy(player, dealer_hand):
    pass

def calculate_hand_value(hand):
    pass

def determine_winner(player, dealer_hand):
    pass

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [71]:
import random

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def value(self):
        if self.rank in ['10', 'J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        else:
            return int(self.rank)

    def __str__(self):
        return f"{self.rank}{self.suit}"

class Deck:
    def __init__(self, num_decks=6):
        suits = ['♠', '♥', '♦', '♣']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.cards = [Card(suit, rank) for _ in range(num_decks) for suit in suits for rank in ranks]
        random.shuffle(self.cards)
        self.plastic_card_position = random.randint(15, 30)

    def draw(self):
        if len(self.cards) < self.plastic_card_position:
            random.shuffle(self.cards)
        return self.cards.pop()

class Player:
    def __init__(self, name, chips):
        self.name = name
        self.chips = chips
        self.hand = []

    def bet(self, amount):
        if amount <= self.chips:
            self.chips -= amount
            return amount
        else:
            return 0

    def win(self, amount):
        self.chips += amount

    def add_card(self, card):
        self.hand.append(card)

    def clear_hand(self):
        self.hand = []

    def strategy(self, dealer_hand):
        pass

class BlackjackGame:
    def __init__(self, num_decks=6):
        self.deck = Deck(num_decks)
        self.players = []
        self.dealer_hand = []

    def add_player(self, player):
        self.players.append(player)

    def deal_initial_cards(self):
        for _ in range(2):
            for player in self.players:
                player.add_card(self.deck.draw())
            self.dealer_hand = [self.deck.draw()]

    def play_round(self):
        for player in self.players:
            while player.strategy(self.dealer_hand) == 'hit':
                player.add_card(self.deck.draw())
        self.dealer_play()
        for player in self.players:
            determine_winner(player, self.dealer_hand)

    def dealer_play(self):
        while calculate_hand_value(self.dealer_hand) < 17:
            self.dealer_hand.append(self.deck.draw())

def player_strategy(player, dealer_hand):
    player_value = calculate_hand_value(player.hand)
    if player_value < 17:
        return 'hit'
    elif player_value >= 17:
        return 'stand'

def calculate_hand_value(hand):
    value = sum(card.value() for card in hand)
    aces = sum(1 for card in hand if card.rank == 'A')
    while value > 21 and aces:
        value -= 10
        aces -= 1
    return value

def determine_winner(player, dealer_hand):
    player_value = calculate_hand_value(player.hand)
    dealer_value = calculate_hand_value(dealer_hand)

    if player_value > 21:
        return "Dealer"
    elif dealer_value > 21 or player_value > dealer_value:
        player.win(player.bet(2))
        return player.name
    elif player_value == dealer_value:
        player.win(player.bet(1))
        return "Push"
    else:
        return "Dealer"

if __name__ == "__main__":
    game = BlackjackGame(num_decks=6)
    player1 = Player("Player 1", chips=1000)
    player2 = Player("Player 2", chips=1000)
    game.add_player(player1)
    game.add_player(player2)

    num_rounds = 1000
    for _ in range(num_rounds):
        game.deck = Deck(num_decks=6)  # Reshuffle decks as needed
        game.deal_initial_cards()
        game.play_round()
        for player in game.players:
            player.clear_hand()

    print(f"Player 1's chips: {player1.chips}")
    print(f"Player 2's chips: {player2.chips}")

Player 1's chips: 1000
Player 2's chips: 1000


5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

In [74]:
if __name__ == "__main__":
    num_dealer_players = 2
    num_rounds = 10

    # Create a BlackjackGame with 2 dealer players
    game = BlackjackGame(num_decks=6)
    dealer1 = Player("Dealer 1", chips=10000)  # Dealer player 1
    dealer2 = Player("Dealer 2", chips=10000)  # Dealer player 2
    game.add_player(dealer1)
    game.add_player(dealer2)

    for round_num in range(1, num_rounds + 1):
        print(f"Round {round_num}:")

        # Create and add human players (you can add more players here)
        player1 = Player("Player 1", chips=1000)
        player2 = Player("Player 2", chips=1000)
        game.add_player(player1)
        game.add_player(player2)

        # Play the round
        game.deck = Deck(num_decks=6)  # Reshuffle decks as needed
        game.deal_initial_cards()
        game.play_round()

        # Show player chips after the round
        for player in game.players:
            print(f"{player.name}'s chips: {player.chips}")

        # Clear hands for the next round
        for player in game.players:
            player.clear_hand()

        # Remove human players from the game
        game.players = game.players[:num_dealer_players]

    print("Game over.")


Round 1:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 2:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 3:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 4:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 5:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 6:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 7:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 8:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 9:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chips: 1000
Player 2's chips: 1000
Round 10:
Dealer 1's chips: 10000
Dealer 2's chips: 10000
Player 1's chip

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

In [77]:
class Player:
    def __init__(self, strategy="dealer", threshold=0):
        # Initialization method with default strategy set to "dealer".
        self.strategy = strategy  # The strategy this player follows: "dealer" or "counting"
        self.threshold = threshold  # The count value at which player decides to hit or stay
        self.cards = []  # Cards currently with the player
        self.card_sum = 0  # Sum of card values currently with the player

    def decision(self):
        # Determines whether the player should "hit" or "stay" based on their strategy.
        if self.strategy == "dealer":
            # If strategy is dealer, hit if sum is less than 17.
            if sum_card_values(self.cards) < 17:
                return "hit"
            return "stay"
        elif self.strategy == "counting":
            # If strategy is counting, decide based on current card count.
            count = self.get_count()
            if count <= self.threshold:
                return "hit"
            return "stay"

    def get_count(self):
        # Computes the card count based on the cards seen so far.
        count = 0
        for card in self.cards:
            if card in ["2", "3", "4", "5", "6"]:
                count += 1
            elif card in ["10", "J", "Q", "K", "A"]:
                count -= 1
        return count

    def take_card(self, card):
        # The player takes a card and updates their card sum.
        self.cards.append(card)
        self.card_sum += card_value(card)

    def reset(self):
        # Resets the player's cards and sum for a new round.
        self.cards = []
        self.card_sum = 0

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

In [80]:
import random

class Player:
    def __init__(self, strategy="dealer", threshold=0):
        self.strategy = strategy
        self.threshold = threshold
        self.cards = []
        self.chips = 100

    def reset(self):
        self.cards = []

    def take_card(self, card):
        self.cards.append(card)

    def decision(self):
        """Decide whether to hit or stay based on strategy."""
        if self.strategy == "dealer":
            return "hit" if sum_card_values(self.cards) < 17 else "stay"
        elif self.strategy == "counting":
            return "hit" if sum_card_values(self.cards) < (12 + self.threshold) else "stay"
        return "stay"

def card_value(card):
    """Returns the Blackjack value of a card."""
    if card in ["2", "3", "4", "5", "6", "7", "8", "9"]:
        return int(card)
    elif card in ["10", "J", "Q", "K"]:
        return 10
    else:  # Ace
        return 11

def sum_card_values(cards):
    """Returns the total value of a hand, adjusting for Aces."""
    s = sum(card_value(card) for card in cards)
    num_aces = cards.count("A")
    while s > 21 and num_aces:
        s -= 10
        num_aces -= 1
    return s

def play_round(players, dealer, verbose=False):
    """Plays a single round of Blackjack."""
    # Initialize deck and shuffle
    deck = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"] * 4
    random.shuffle(deck)

    # Deal initial cards
    for player in players:
        player.reset()
        player.take_card(deck.pop())
        player.take_card(deck.pop())
    dealer.reset()
    dealer.take_card(deck.pop())
    dealer.take_card(deck.pop())

    # Players' turns
    for player in players:
        while player.decision() == "hit":
            player.take_card(deck.pop())
            if sum_card_values(player.cards) > 21:
                break  # Player busts

    # Dealer's turn
    while dealer.decision() == "hit":
        dealer.take_card(deck.pop())

    # Compute results
    dealer_sum = sum_card_values(dealer.cards)
    for player in players:
        player_sum = sum_card_values(player.cards)
        if verbose:
            print(f"Player ({player.strategy}) total: {player_sum}")

        # Win/loss conditions
        if player_sum > 21 or (dealer_sum <= 21 and dealer_sum >= player_sum):
            player.chips -= 1  # Loss
        else:
            player.chips += 1  # Win

    if verbose:
        print(f"Dealer total: {dealer_sum}")

def test_scenario():
    """Test scenario where one player uses counting strategy."""
    counting_player = Player(strategy="counting", threshold=-2)
    dealer = Player(strategy="dealer")
    players = [counting_player, Player(strategy="dealer"), Player(strategy="dealer"), Player(strategy="dealer")]

    # Initialize chips
    for player in players:
        player.chips = 100

    # Play 50 rounds
    for _ in range(50):
        play_round(players, dealer)

    print(f"Counting player's chips after 50 rounds: {counting_player.chips}")

test_scenario()


Counting player's chips after 50 rounds: 96


8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


In [83]:
def multiple_games(num_games=100):
    """Simulate multiple games and track winnings of the counting player."""
    winnings = []

    for _ in range(num_games):
        # Initialize players and dealer
        counting_player = Player(strategy="counting", threshold=-2)
        dealer = Player(strategy="dealer")
        players = [counting_player, Player(strategy="dealer"), Player(strategy="dealer"), Player(strategy="dealer")]

        # Set initial chip counts
        for player in players:
            player.chips = 100

        # Play 50 rounds
        for _ in range(50):
            play_round(players, dealer, verbose=False)

        # Store final chip count of counting player
        winnings.append(counting_player.chips)

    return winnings

# Run the simulation
winnings = multiple_games()
print(winnings)  # Optional: Print results to analyze trends


[88, 80, 86, 76, 88, 76, 102, 92, 90, 90, 92, 90, 92, 76, 92, 86, 94, 84, 86, 90, 94, 90, 82, 82, 100, 94, 100, 84, 84, 82, 88, 92, 78, 96, 86, 82, 100, 86, 90, 88, 98, 90, 90, 82, 80, 94, 98, 98, 96, 82, 98, 72, 96, 78, 104, 84, 96, 90, 86, 88, 82, 88, 86, 90, 88, 90, 84, 86, 94, 84, 80, 96, 86, 88, 102, 92, 84, 98, 84, 86, 86, 84, 94, 98, 94, 78, 96, 92, 84, 92, 90, 84, 94, 80, 86, 94, 86, 88, 90, 78]


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

In [86]:
import numpy as np

def multiple_games_with_threshold(num_games=100, threshold=-2):
    """Simulate multiple games for a specific threshold and track winnings."""
    winnings = []
    
    for _ in range(num_games):
        counting_player = Player(strategy="counting", threshold=threshold)
        dealer = Player(strategy="dealer")
        players = [counting_player, Player(strategy="dealer"), Player(strategy="dealer"), Player(strategy="dealer")]

        for player in players:
            player.chips = 100

        for _ in range(50):
            play_round(players, dealer, verbose=False)

        winnings.append(counting_player.chips)
    
    return winnings

def test_thresholds():
    """Test different counting strategy thresholds and analyze performance."""
    thresholds = [-4, -2, 0, 2, 4]
    results = {t: multiple_games_with_threshold(threshold=t) for t in thresholds}
    
    return results

# Run the test
threshold_results = test_thresholds()

# Print the average winnings for each threshold
print("\nThreshold Performance Summary:")
for t, res in threshold_results.items():
    print(f"Threshold {t}: Average winnings = {np.mean(res):.2f}")



Threshold Performance Summary:
Threshold -4: Average winnings = 88.72
Threshold -2: Average winnings = 89.42
Threshold 0: Average winnings = 91.44
Threshold 2: Average winnings = 91.02
Threshold 4: Average winnings = 92.30


10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

In [89]:
import random

def dice_roll():
    """Simulates rolling a 6-sided die."""
    return random.randint(1, 6)

def bet(balance, bet_amount):
    """Places a bet and updates balance based on dice roll outcome."""
    roll = dice_roll()
    if roll <= 3:  # Win if roll is 1, 2, or 3 (50% chance)
        return balance + bet_amount
    else:
        return balance - bet_amount

def play_game(initial_balance, max_multiplier=2):
    """
    Simulates a betting game where:
    - You start with `initial_balance`
    - You bet until balance reaches 0 or `max_multiplier * initial_balance`
    """
    balance = initial_balance
    rounds = 0
    max_balance = initial_balance * max_multiplier

    while balance > 0 and balance < max_balance:
        bet_amount = max(1, min(balance, initial_balance // 2))  # Ensure at least $1 bet
        balance = bet(balance, bet_amount)
        rounds += 1

    return balance, rounds

if __name__ == "__main__":
    initial_balance = 1000
    final_balance, rounds_played = play_game(initial_balance)

    if final_balance >= 2 * initial_balance:
        print(f"You won! Your final balance is ${final_balance}.")
    else:
        print(f"You lost. Your final balance is ${final_balance}.")
    
    print(f"Rounds played: {rounds_played}")


You lost. Your final balance is $0.
Rounds played: 2
